In [ ]:
# https://gseapy.readthedocs.io/en/latest/gseapy_example.html

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp

In [2]:
enrichr_adj_pval_thresh = 0.05

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
dbs = gp.get_library_name()


In [5]:
go_bp = gp.get_library(name = 'GO_Biological_Process_2023', organism = 'human')

## Read in naming file

In [6]:
inputFile = 'data/omics_revamped_LLM_DF.tsv'

In [7]:
outputFile = 'data/omics_revamped_LLM_Enrichr_DF.tsv' 

In [8]:
genesets_LLM = pd.read_csv(inputFile, sep = "\t") 

In [9]:
# genesets_LLM.GeneSetID.unique()

In [10]:
# Remove "HALLMARK_" prefix for MSigDB gene sets
genesets_LLM['GeneSetName'] = genesets_LLM['GeneSetName'].str.removeprefix("HALLMARK_").str.replace('_', ' ')


In [11]:
genesets_LLM.columns

Index(['Source', 'GeneSetID', 'GeneSetName', 'GeneList', 'n_Genes', 'LLM Name',
       'LLM Analysis', 'Score'],
      dtype='object')

In [12]:
columnsToKeep = ['Source', 'GeneSetID', 'GeneSetName', 'GeneList', 'n_Genes', 'LLM Name',
       'LLM Analysis', 'Score']
geneSeparator = " "

In [13]:
LLM_DF = genesets_LLM

In [14]:
nGeneSets = LLM_DF.shape[0]

In [15]:
nGeneSets

300

In [16]:
LLM_repeated_DF = LLM_DF[LLM_DF[['Source', 'GeneSetID', 'GeneSetName', 'GeneList']].duplicated() == True]


In [17]:
LLM_DF.shape

(300, 8)

## Go through each system and run Enrichr 

In [18]:
# Each system will have multiple rows

In [19]:
additionalCols = ['Rank', 'Overlap', 'P-value', 'Adjusted P-value', 'Genes', 'Term', 'GO term', 'GO ID' ]

In [20]:
expandedColumnsToKeep =columnsToKeep;
expandedColumnsToKeep.extend(additionalCols)

In [21]:
expanded_LLM_genes_DF = pd.DataFrame(columns=expandedColumnsToKeep)

In [22]:
genesCol = 'GeneList'

In [23]:
for geneSetInd in range(nGeneSets):
    print(geneSetInd)
    geneSetRow = LLM_DF.iloc[geneSetInd]
    geneSetGenes = geneSetRow[genesCol].split(geneSeparator)

    # Perform enrichment analysis
    enr = gp.enrichr(gene_list=geneSetGenes, # or "./tests/data/gene_list.txt",
                 gene_sets=['GO_Biological_Process_2023'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )
    
    # threshold
    # sigRes_DF =  enr.results.loc[enr.results['Adjusted P-value'] <=0.1]
    sigRes_DF = enr.results # remove thresholding, because want a name for each one
    sigRes_DF = sigRes_DF.sort_values('Adjusted P-value')
    if sigRes_DF.shape[0] == 0:
        
        print([geneSetInd, ' has not enirchment'])
        #continue # skip to next gene set
        sigRes_DF.loc[0, ['Term', 'GO term', 'GO ID', 'nothing']] = 'NaN'
        sigRes_DF.loc[0, ['Rank']]  = 1
        sigRes_DF.loc[0, ['Overlap']]  = 0
        sigRes_DF.loc[0, ['P-value']]  = 1
        sigRes_DF.loc[0, ['Adjusted P-value']]  = 1
        sigRes_DF.loc[0, ['Genes']]  = ''
        
    else:
        expandedCols = sigRes_DF["Term"].str.split('(', expand = True)
        
        if expandedCols.shape[1] == 3:
            sigRes_DF[['GO term', 'GO ID', 'nothing']] = expandedCols
        else: # ==2
            sigRes_DF[['GO term', 'GO ID']] = expandedCols

        sigRes_DF[['GO ID', 'nothing']] = sigRes_DF["GO ID"].str.split(')', expand = True)
        

        # Get ranking of enriched GO terms
        sigRes_DF['Rank'] = sigRes_DF.reset_index().index

        if sigRes_DF.loc[0, 'Adjusted P-value'] > enrichr_adj_pval_thresh: # nothing significant
            sigRes_DF = sigRes_DF.loc[[0],:]
        else: # there are sig. terms
            sigRes_DF = sigRes_DF[sigRes_DF['Adjusted P-value'] <=enrichr_adj_pval_thresh]
        
            
        
    
    # Doing what should be a simple colbind 
    geneSetRow_DF = geneSetRow.to_frame().T
    geneSetRow_repeated_DF = geneSetRow_DF.loc[geneSetRow_DF.index.repeat(sigRes_DF.shape[0])]
    geneSetRow_repeated_DF = geneSetRow_repeated_DF.reset_index()
    
    # system_Row_sigRes = systemRow_repeated_DF.join(sigRes_DF)
    geneSetRowRow_sigRes = pd.concat([geneSetRow_repeated_DF, sigRes_DF],axis=1)
    
    # Doing what should be a simple  rowbind
    if expanded_LLM_genes_DF.shape[0] == 0:
        expanded_LLM_genes_DF = geneSetRowRow_sigRes[expandedColumnsToKeep]
    else:
        expanded_LLM_genes_DF =  pd.concat([expanded_LLM_genes_DF, geneSetRowRow_sigRes[expandedColumnsToKeep]],axis=0)
    
    if (geneSetInd%10 == 1):
        expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t") # Save to not loose work

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
def get_goterm_genes(goterm):
    if goterm in go_bp:
        return " ".join(go_bp[goterm])
    else:
        return ""

In [25]:
expanded_LLM_genes_DF['GO_term_genes'] = [get_goterm_genes(goterm) for goterm in expanded_LLM_genes_DF['Term']]

In [26]:
expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t") # Too large 

In [27]:
expanded_LLM_genes_DF.shape

(11310, 17)

In [28]:
expanded_LLM_genes_copy_DF = expanded_LLM_genes_DF;

In [29]:
def select_min_rows(df, group_vars, min_var):
    # Convert the target column to numeric, non-numeric values become NaN
    df[min_var] = pd.to_numeric(df[min_var], errors='coerce')

    # Reset index of the DataFrame
    df_reset = df.reset_index()

    # Get the indices of the minimum values in the grouped DataFrame
    min_indices = df_reset.groupby(group_vars)[min_var].idxmin()

    # Use these indices to select rows from the reset DataFrame
    return df_reset.loc[min_indices].set_index('index')

In [30]:
reduced_LLM_genes_DF = select_min_rows(expanded_LLM_genes_copy_DF, ['Source', 'GeneSetID', 'GeneSetName', 'GeneList'], 'Adjusted P-value')

In [31]:
reduced_LLM_genes_DF.shape

(300, 17)

In [32]:
reduced_LLM_genes_DF.to_csv('data/omics_revamped_LLM_Enrichr_reduced_DF.tsv' , sep = "\t")

In [33]:
set(LLM_DF.GeneSetID.to_list()).difference(set(reduced_LLM_genes_DF.GeneSetID.to_list()))

set()

In [34]:
reduced_LLM_genes_DF[reduced_LLM_genes_DF[['Source', 'GeneSetID', 'GeneSetName', 'GeneList']].duplicated() == True]


,Source,GeneSetID,GeneSetName,GeneList,n_Genes,LLM Name,LLM Analysis,Score,Rank,Overlap,P-value,Adjusted P-value,Genes,Term,GO term,GO ID,GO_term_genes
index,,,,,,,,,,,,,,,,,


In [ ]:
# reduced_LLM_genes_DF = expanded_LLM_genes_DF.loc[expanded_LLM_genes_DF.reset_index().groupby(['Source', 'GeneSetID', 'GeneSetName', 'GeneList'])['Adjusted P-value'].idxmin()]

In [ ]:
reduced_LLM_genes_DF